In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from gensim.models import Word2Vec

2024-03-22 15:43:53.346183: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 15:43:53.346324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 15:43:53.471298: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### **Data Pipeline**

In [2]:
def getLexedArrays(path, size=-1):
    array = []
    with open(path, 'r') as f:
        i = 0
        while size == -1 or i < size:
            line = f.readline()
            if line:
                array.append(line.split())
                i += 1
            else:
                break
    return array


def getLabels(path, size=-1):
    array = []
    with open(path, 'r') as f:
        i = 0
        while size == -1 or i < size:
            line = f.readline()
            if line:
                array.append(int(line))
                i += 1
            else:
                break
    return array


In [3]:
test_arrays = getLexedArrays('../input/lexeds-vdisc/lexed_VDISC/VDISC_test', 50000)
train_arrays = getLexedArrays('../input/lexeds-vdisc/lexed_VDISC/VDISC_train', 500000)
val_arrays = getLexedArrays('../input/lexeds-vdisc/lexed_VDISC/VDISC_validate', 50000)

test_labels = getLabels('../input/labels-vdisc/labels_VDISC/VDISC_test', 50000)
train_labels = getLabels('../input/labels-vdisc/labels_VDISC/VDISC_train', 500000)
val_labels = getLabels('../input/labels-vdisc/labels_VDISC/VDISC_validate', 50000)

#### **Obtain Maximum Array Length for Padding**

In [4]:
max_len = 0
for lexed_arrays in (test_arrays, train_arrays, val_arrays):
    for ar in lexed_arrays:
        max_len = max(max_len, len(ar))

#### **Create Tensorflow Datasets from Generator**

In [16]:
VEC_DIM = 15
def data_generator(dataset, labels):
    w2vmodel = Word2Vec.load("../input/modelll/embedding.model")
    for j, ar in enumerate(dataset):
        vec_ems = np.zeros((max_len, VEC_DIM))

        for i, k in enumerate(ar):
            vec = None
            if k not in w2vmodel.wv:
                vec = w2vmodel.wv['<|na|>']
            else:
                vec = w2vmodel.wv[k]
            vec_ems[i, :] = vec
            
        shape = vec_ems.shape
        
        yield (vec_ems, labels[j] if labels[j] != -1 else 0)

test_ds = tf.data.Dataset.from_generator(
    lambda: data_generator(test_arrays, test_labels),
    output_types=(tf.float32, tf.float32), 
    output_shapes=((max_len, VEC_DIM), ()),
)

train_ds = tf.data.Dataset.from_generator(
    lambda: data_generator(train_arrays, train_labels),
    output_types=(tf.float32, tf.float32), 
    output_shapes=((max_len, VEC_DIM), ()),
)

val_ds = tf.data.Dataset.from_generator(
    lambda: data_generator(val_arrays, val_labels),
    output_types=(tf.float32, tf.float32), 
    output_shapes=((max_len, VEC_DIM), ()),
)

AUTOTUNE = tf.data.AUTOTUNE
batch_size = 64

train_ds = train_ds.repeat(10).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

### **Declaring Model and Hyperparameters**

In [17]:
NUM_FILTERS = 32

model = Sequential([
    layers.Input(shape=(max_len, VEC_DIM)),
    layers.Conv1D(filters=NUM_FILTERS, kernel_size=15, activation='relu'),
    layers.GlobalMaxPooling1D(),
    layers.Dropout(0.7),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.7),
    layers.Dense(128, activation='relu'),
    layers.Dense(6)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 981, 32)        │         7,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_3          │ (None, 32)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,742 (112.27 KB)

 Trainable params: 28,742 (112.27 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs=1
history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=epochs
)

    230/Unknown 19s 63ms/step - accuracy: 0.8925 - loss: 0.5418

In [ ]:
w2vmodel = Word2Vec.load("../input/modelll/embedding.model")
w2vmodel.wv.index_to_key